In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
#from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from helper import *

main = pd.read_pickle("../Data/main_no_outliers.df")

TARGET = "lengths_behind"
FEATURES =  ["days_since_last_race_rel", "horse_record_rel", "jockey_record_rel", "trainer_record_rel", 
             "horse_rating_rel", "win_percent_rel", "declared_weight_rel", 
             "actual_weight_rel", "last_race_result_rel", "last_refined_score_rel", 
             "last_raw_score_rel", "mean_raw_score_rel", "mean_ref_score_rel", "best_horse_record", 
             "best_going_record", "best_jockey_record", "best_trainer_record", 
             "best_win_percent"
]
# FEATURES =  ["days_since_last_race", "horse_record", "jockey_record", "trainer_record", 
#              "horse_rating", "win_percent", "declared_weight", 
#              "actual_weight", "last_race_result", "last_refined_score", 
#              "last_raw_score", "mean_raw_score", "mean_ref_score", "best_horse_record", 
#              "best_going_record", "best_jockey_record", "best_trainer_record", 
#              "best_win_percent"
# ]

In [2]:
def lengthsWinEval(model, X_train, y_train, X_test, y_test, testing):
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    testing = testing.reset_index()
    
    # RMSE eval:
    RMSE = mean_squared_error(preds, y_test, squared=False)
    print("RMSE = {:.3f}".format(RMSE))
    
    races = np.unique(testing["race_id"])
    
    # Now we rank accordingly
    count = 0
    winCount = 0
    for race_id in races:
        
        raceI = list(testing.loc[testing["race_id"]==race_id].index)
        
        temp_preds = preds[raceI]
        temp_act = y_test.iloc[raceI]
        
        predWinner = np.argmin(temp_preds, axis=0)
        actWinner = np.argmin(temp_act, axis=0)
        
        if predWinner == actWinner:
            winCount += 1
            
    print("Win percent = {:.3f}".format(float(winCount)/len(races)))

In [3]:
plt.figure(figsize=(10,8), dpi=100)
for training, testing in getCrossVal(main, 5):
    #model = MLPRegressor(hidden_layer_sizes=(int(len(FEATURES)/2)), solver='lbfgs', max_iter=4000)
    model = GradientBoostingRegressor()
    X_train = training[FEATURES]
    y_train = training[TARGET]
    X_test = testing[FEATURES]
    y_test = testing[TARGET]
    
    lengthsWinEval(model, X_train, y_train, X_test, y_test, testing)
    
plt.show()

RMSE = 4.253
Win percent = 0.212
RMSE = 4.197
Win percent = 0.207
RMSE = 4.154
Win percent = 0.203
RMSE = 4.246
Win percent = 0.224


KeyboardInterrupt: 